In [1]:
import numpy as np
import pandas as pd
import re
import numpy

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)
    
pd_atp_matches = pd.read_csv('tennis_atp/atp_matches_2017.csv')
my_data_stats = pd.read_csv('data/match_stats_2017_UNINDEXED.csv')

#select what do I want from the stats dataset
mycolumns=[
           'ms_winner_aces','ms_loser_aces','ms_winner_double_faults','ms_loser_double_faults',
           'ms_winner_first_serves_total','ms_winner_first_serve_points_won','ms_winner_first_serves_in',
           'ms_loser_first_serves_total','ms_loser_first_serve_points_won','ms_loser_first_serves_in',
           'ms_winner_return_points_won','ms_winner_return_points_total',
           'ms_loser_return_points_won','ms_loser_return_points_total',
           'ms_winner_total_points_won','ms_winner_total_points_total',
            'ms_winner_second_serve_points_total','ms_loser_second_serve_points_total']
new_columns=[
             'w_ace','l_ace','w_df','l_df',
             'w_svpt','w_1stWon','w_1stIn',
             'l_svpt','l_1stWon','l_1stIn',   
             'w_ret_w','w_ret_tot',
             'l_ret_w','l_ret_tot',
             'w_pts_w','total_points',
            'w_2ndTot','l_2ndTot']

my_data_stats=my_data_stats.filter(mycolumns, axis=1)
my_data_stats.columns = new_columns

#select what I want from the other dataset
labels=['tourney_id','tourney_name','round','winner_id','winner_name','loser_id','loser_name','score',
        'winner_rank','winner_rank_points','loser_rank','loser_rank_points',
        'w_ace','l_ace','w_df','l_df',
        'w_svpt','w_1stWon','w_1stIn',
        'l_svpt','l_1stWon','l_1stIn',
        'w_2ndWon','l_2ndWon',
        'w_bpSaved','w_bpFaced',
        'l_bpSaved','l_bpFaced']
pd_atp_matches=pd_atp_matches.filter(labels, axis=1)

labels_match=['w_ace','l_ace','w_df','l_df',
                'w_svpt','w_1stWon','w_1stIn',
                'l_svpt','l_1stWon','l_1stIn']
pd_atp_matches=pd.merge(pd_atp_matches,my_data_stats,on=labels_match).reset_index(drop=True)
pd_atp_matches=pd_atp_matches.drop_duplicates(keep='first').reset_index(drop=True)


#calculate number of games
def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

def change_char(s, p, r):
    return s[:p]+r+s[p+1:]

pd_atp_matches=pd_atp_matches.assign(TotGames=0)

#score to number of games
for i in range(0,len(pd_atp_matches.index)):
    str1=pd_atp_matches.loc[i, 'score']
    
    start=findOccurrences(str1, '(')
    end=findOccurrences(str1, ')')

    for k in range(0,len(start)):
        for l in range(0,end[k]-start[k]+1):
            str1=change_char(str1,start[k]+l,' ')

    number_of_games=0
    for j in range (0,len(list(filter(str.isdigit, str1)))):
        number_of_games=number_of_games+int(list(filter(str.isdigit, str1))[j])
    pd_atp_matches.loc[i, 'TotGames'] = number_of_games

    
def SetNan(pandas,column):
    list = pandas.index[(pandas[column].isin([0])) == True].tolist()
    for i in range(0,len(list)):
        pandas.loc[list[i],column]=np.nan
        
SetNan(pd_atp_matches,'w_svpt')
SetNan(pd_atp_matches,'l_svpt')
SetNan(pd_atp_matches,'w_1stIn')
SetNan(pd_atp_matches,'l_1stIn')
SetNan(pd_atp_matches,'w_2ndTot')
SetNan(pd_atp_matches,'l_2ndTot')

In [2]:
#this function returns WRP of a player against a certain adversary

def getMatchesWonPer(myopponent,adversary):
    exp1=pd_atp_matches['winner_id'].isin([myopponent])
    exp2=pd_atp_matches['loser_id'].isin([adversary])
    exp3=pd_atp_matches['loser_id'].isin([myopponent])
    exp4=pd_atp_matches['winner_id'].isin([adversary])

    matches_won=len(pd_atp_matches.loc[(exp1&exp2)].index)
    matches_lost=len(pd_atp_matches.loc[(exp3&exp4)].index)
    matchesWonPer = matches_won/(matches_won+matches_lost)
    
    return matchesWonPer

pd_atp_matches=pd_atp_matches.assign(TMW=0)
for i in range(0,len(pd_atp_matches.index)):
    TMW1=getMatchesWonPer(pd_atp_matches.loc[i, 'winner_id'],pd_atp_matches.loc[i, 'loser_id'])
    pd_atp_matches.loc[i, 'TMW'] = TMW1 - (1-TMW1)
    

In [3]:
display(pd_atp_matches.loc[467])

tourney_id                    2017-0495
tourney_name                      Dubai
round                                SF
winner_id                        104269
winner_name           Fernando Verdasco
loser_id                         104898
loser_name                  Robin Haase
score                    7-6(5) 5-7 6-1
winner_rank                          35
winner_rank_points                 1070
loser_rank                           66
loser_rank_points                   763
w_ace                                 2
l_ace                                 8
w_df                                  1
l_df                                  1
w_svpt                               85
w_1stWon                             54
w_1stIn                              73
l_svpt                              113
l_1stWon                             42
l_1stIn                              66
w_2ndWon                              6
l_2ndWon                             22
w_bpSaved                             6


In [5]:
results = pd_atp_matches.filter(['winner_name','loser_name'], axis=1)

#Ranking Won
results['RANK'] = pd_atp_matches['winner_rank'] - pd_atp_matches['loser_rank']
results['POINTS'] = pd_atp_matches['winner_rank_points'] - pd_atp_matches['loser_rank_points']

#First Serve In
FS1 = pd_atp_matches['w_1stIn'].div(pd_atp_matches['w_svpt']) 
FS2 =  pd_atp_matches['l_1stIn'].div(pd_atp_matches['l_svpt'])
results['FS'] = FS1 - FS2

#Winning in First Serve
W1SP1 = pd_atp_matches['w_1stWon'].div(pd_atp_matches['w_1stIn'])
W1SP2 =  pd_atp_matches['l_1stWon'].div(pd_atp_matches['l_1stIn'])
results['W1SP'] = W1SP1 - W1SP2

#Winning in Second Serve
W2SP1 = pd_atp_matches['w_2ndWon'].div(pd_atp_matches['w_2ndTot'])
W2SP2 =  pd_atp_matches['l_2ndWon'].div(pd_atp_matches['l_2ndTot'])
results['W2SP'] = W2SP1 - W2SP2

#Winning on Serve Cumulative
WSP1 = W1SP1 * FS1 + W2SP1 * (1-FS1)
WSP2 = W1SP2 * FS2 + W2SP2 * (1-FS2)
results['WSP'] = WSP1 - WSP2

#Winning in Return
WRP1 = pd_atp_matches['w_ret_w'].div(pd_atp_matches['w_ret_tot'])
WRP2 =  pd_atp_matches['l_ret_w'].div(pd_atp_matches['l_ret_tot'])
results['WRP'] = WRP1 - WRP2

#Total Points Won
TPW1 = pd_atp_matches['w_pts_w']/pd_atp_matches['total_points']
TPW2 = (pd_atp_matches['total_points']-pd_atp_matches['w_pts_w'])/pd_atp_matches['total_points']
results['TPW'] = TPW1 - (1-TPW1)

results['TMW'] = pd_atp_matches['TMW']

results['ACES'] = (pd_atp_matches['w_ace']-pd_atp_matches['l_ace'])/pd_atp_matches['TotGames']
results['DF'] = (pd_atp_matches['w_df']-pd_atp_matches['l_df'])/pd_atp_matches['TotGames']

BP1 = pd_atp_matches['w_bpSaved']/pd_atp_matches['w_bpFaced']
BP2 = pd_atp_matches['l_bpSaved']/pd_atp_matches['l_bpFaced']
results['BP'] = BP1-BP2

COMPLETE1 = WSP1 * WRP1
COMPLETE2 = WSP2 * WRP2
results['COMPLETE'] = COMPLETE1-COMPLETE2

SERVEADV1 = WSP1 - WRP2
SERVEADV2 = WSP2 - WRP1
results['SERVEADV'] = SERVEADV1-SERVEADV2

#results.rename(columns{'winner_id': 'ID1', 'loser_id': 'ID2'}, inplace=True)

results=results.assign(winner=1)

results.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]

,winner_name,loser_name,RANK,POINTS,FS,W1SP,W2SP,WSP,WRP,TPW,TMW,ACES,DF,BP,COMPLETE,SERVEADV,winner
0,Grigor Dimitrov,Kei Nishikori,12.0,-2870.0,-0.0348203,0.0537677,0.030000,0.0361378,0.036138,0.054795,1.000000,0.12,0.08,0.314286,0.0361378,0.0722756,1
1,Grigor Dimitrov,Milos Raonic,14.0,-3415.0,0.161673,-0.107143,0.333333,0.120124,0.120124,0.109244,1.000000,0,-0.0952381,NaN,0.120124,0.240249,1
2,Kei Nishikori,Stanislas Wawrinka,1.0,-410.0,0.120715,-0.0690154,0.250000,0.0557803,0.055780,0.086957,1.000000,-0.363636,-0.0454545,0.800000,0.0557803,0.111561,1
3,Milos Raonic,Rafael Nadal,-6.0,2150.0,-0.0870152,0.167107,-0.151553,0.04946,0.049460,0.071823,1.000000,0.655172,0.103448,0.357143,0.04946,0.09892,1
4,Grigor Dimitrov,Dominic Thiem,9.0,-1380.0,0.101972,0.0239085,0.014286,0.0451479,0.045148,0.068182,0.333333,-0.107143,-0.0714286,0.357143,0.0451479,0.0902958,1
5,Kei Nishikori,Jordan Thompson,-74.0,4216.0,-0.0369212,0.411654,0.403509,0.397997,0.397997,0.358025,1.000000,0.142857,-0.142857,NaN,0.397997,0.795995,1
9,Dominic Thiem,Samuel Groth,-172.0,3095.0,0.0734748,-0.153247,0.322981,0.0432361,0.043236,0.073171,1.000000,0,0,0.250000,0.0432361,0.0864721,1
1000,Novak Djokovic,Nicolas Almagro,-74.0,6410.0,0.151564,-0.0893985,0.289803,0.0961202,0.096120,0.083799,1.000000,-0.137931,-0.0344828,0.025974,0.0961202,0.19224,1
1001,Marius Copil,Guillermo Garcia Lopez,-20.0,70.0,0.116745,0.0277778,0.023109,0.0529186,0.052919,0.017094,1.000000,0.0277778,-0.111111,-0.100000,0.0529186,0.105837,1
1002,Borna Coric,Mischa Zverev,28.0,-344.0,-0.198871,0.0329268,0.489011,0.151674,0.151674,0.134752,1.000000,0.181818,-0.136364,0.083333,0.151674,0.303348,1


In [6]:
#Now order according to ranking to see how good they are

features = results.filter(['RANK','POINTS','FS','W1SP','W2SP','WSP','WRP','TPW','TMW','ACES','DF','BP','COMPLETE','SERVEADV','winner'], axis=1)

features=features.assign(player1=' ')
features=features.assign(player2=' ')

for i in range(0,len(features.index)):
    if(features.loc[i, 'RANK']<0):
        features.loc[i, 'player1']=results.loc[i,'winner_name']
        features.loc[i, 'player2']=results.loc[i,'loser_name']
    else:
        features.loc[i, 'player1']=results.loc[i,'loser_name']
        features.loc[i, 'player2']=results.loc[i,'winner_name']
        features.loc[i, 'RANK':'SERVEADV']=features.loc[i, 'RANK':'SERVEADV']*-1
        features.loc[i, 'winner']=0

features=features.fillna(0)        
features.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]

features.to_csv('newfeatures2017.csv',header='false',index='false')
features=pd.read_csv('newfeatures2017.csv')
features.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]

,Unnamed: 0,RANK,POINTS,FS,W1SP,W2SP,WSP,WRP,TPW,TMW,ACES,DF,BP,COMPLETE,SERVEADV,winner,player1,player2
0,0,-12.0,2870.0,0.034820,-0.053768,-0.030000,-0.036138,-0.036138,-0.054795,-1.000000,-0.120000,-0.080000,-0.314286,-0.036138,-0.072276,0,Kei Nishikori,Grigor Dimitrov
1,1,-14.0,3415.0,-0.161673,0.107143,-0.333333,-0.120124,-0.120124,-0.109244,-1.000000,-0.000000,0.095238,0.000000,-0.120124,-0.240249,0,Milos Raonic,Grigor Dimitrov
2,2,-1.0,410.0,-0.120715,0.069015,-0.250000,-0.055780,-0.055780,-0.086957,-1.000000,0.363636,0.045455,-0.800000,-0.055780,-0.111561,0,Stanislas Wawrinka,Kei Nishikori
3,3,-6.0,2150.0,-0.087015,0.167107,-0.151553,0.049460,0.049460,0.071823,1.000000,0.655172,0.103448,0.357143,0.049460,0.098920,1,Milos Raonic,Rafael Nadal
4,4,-9.0,1380.0,-0.101972,-0.023909,-0.014286,-0.045148,-0.045148,-0.068182,-0.333333,0.107143,0.071429,-0.357143,-0.045148,-0.090296,0,Dominic Thiem,Grigor Dimitrov
5,5,-74.0,4216.0,-0.036921,0.411654,0.403509,0.397997,0.397997,0.358025,1.000000,0.142857,-0.142857,0.000000,0.397997,0.795995,1,Kei Nishikori,Jordan Thompson
9,9,-172.0,3095.0,0.073475,-0.153247,0.322981,0.043236,0.043236,0.073171,1.000000,0.000000,0.000000,0.250000,0.043236,0.086472,1,Dominic Thiem,Samuel Groth
1000,1000,-74.0,6410.0,0.151564,-0.089398,0.289803,0.096120,0.096120,0.083799,1.000000,-0.137931,-0.034483,0.025974,0.096120,0.192240,1,Novak Djokovic,Nicolas Almagro
1001,1001,-20.0,70.0,0.116745,0.027778,0.023109,0.052919,0.052919,0.017094,1.000000,0.027778,-0.111111,-0.100000,0.052919,0.105837,1,Marius Copil,Guillermo Garcia Lopez
1002,1002,-28.0,344.0,0.198871,-0.032927,-0.489011,-0.151674,-0.151674,-0.134752,-1.000000,-0.181818,0.136364,-0.083333,-0.151674,-0.303348,0,Mischa Zverev,Borna Coric
